In [29]:
import nbformat
import os
import re

def create_course_outline(parent_folder, output_notebook="Course_Outline.ipynb"):
    # Initialize a new notebook
    new_nb = nbformat.v4.new_notebook()
    
    lesson_pattern = re.compile(r'^Lesson_\d+_')
    notebook_pattern = re.compile(r'L\d+_\d+_Overview\.ipynb')
    
    topics = []  # Store each lesson's header for the Topics section

    # Process L00_0_Overview.ipynb first, if it exists
    first_notebook_path = os.path.join(parent_folder, 'Lesson_00', 'L00_0_Overview.ipynb')
    if os.path.exists(first_notebook_path):
        with open(first_notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
            for cell in nb.cells:
                if cell.cell_type == 'markdown':
                    # Process the content to exclude "### Homework Ideas" and everything after it
                    content_lines = cell.source.splitlines()
                    modified_lines = []
                    
                    for line in content_lines:
                        if line.startswith("### Homework Ideas"):
                            break
                        modified_lines.append(line)
                    
                    new_nb.cells.append(nbformat.v4.new_markdown_cell(modified_lines))
                    break  # Stop after adding the first markdown cell from L00_0_Overview.ipynb

    # Process the remaining notebooks and add headers to the Topics section
    for folder_name in sorted(os.listdir(parent_folder)):
        # Check if folder matches 'Lesson_XX_{something}'
        if lesson_pattern.match(folder_name):
            folder_path = os.path.join(parent_folder, folder_name)
            
            # Search for the specific notebook 'L_XX_X_Overview.ipynb'
            for file_name in os.listdir(folder_path):
                if notebook_pattern.match(file_name):
                    notebook_path = os.path.join(folder_path, file_name)
                    
                    # Load the notebook and grab the first markdown cell
                    with open(notebook_path, 'r', encoding='utf-8') as f:
                        nb = nbformat.read(f, as_version=4)
                        for cell in nb.cells:
                            if cell.cell_type == 'markdown':
                                # Grab the first header (e.g., '# Lesson 1: Title') for Topics section
                                first_line = cell.source.splitlines()[0]
                                if first_line.startswith("#"):
                                    topics.append(first_line.lstrip('# ').strip())
                                
                                # Process cell content, stopping at "### Homework Ideas"
                                content_lines = cell.source.splitlines()
                                modified_lines = []
                                
                                for line in content_lines:
                                    if line.startswith("### Homework Ideas"):
                                        break
                                    modified_lines.append(line)
                                
                                # Combine the modified lines and add '---' separator
                                modified_content = "\n".join(modified_lines) + "\n\n---"
                                new_nb.cells.append(nbformat.v4.new_markdown_cell(modified_content))
                                break  # Stop after adding the first markdown cell
                    break  # Stop after finding the 'L_XX_X_Overview.ipynb' file in this folder

    # Create the Topics cell with the collected headers and insert it after the first notebook
    topics_content = "## Weekly Topics\n\n" + "\n".join(f"- {topic}" for topic in topics)
    # add '---' separator
    topics_modified = topics_content + "\n\n---"
    new_nb.cells.insert(1, nbformat.v4.new_markdown_cell(topics_modified))

    # Write the new notebook to a file
    with open(output_notebook, 'w', encoding='utf-8') as f:
        nbformat.write(new_nb, f)
    print(f"Course outline created: {output_notebook}")

# Set the parent folder path containing 'Lesson_XX_{something}' folders
parent_folder = '.'  # Replace '.' with the path to the main directory
create_course_outline(parent_folder)


Course outline created: Course_Outline.ipynb
